DENSE RETRIEVAL

In [ ]:
!pip install -U beir

     |████████████████████████████████| 49 kB 4.7 MB/s 
     |████████████████████████████████| 8.4 MB 10.7 MB/s 
     |████████████████████████████████| 378 kB 56.8 MB/s 
     |████████████████████████████████| 4.4 MB 56.5 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 2.9 MB 50.8 MB/s 
     |████████████████████████████████| 3.3 MB 47.4 MB/s 
     |████████████████████████████████| 1.2 MB 72.4 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 
     |████████████████████████████████| 596 kB 71.1 MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=265101 sha256=2c3b037eadcf7864e53ce24a5c7b4890d2c20e2922d5c96239862be1da250a57
  Stored in directory: /root/.cache/pip/wheels/42/96/77/0829b8b2606f90f61ba10a51277629d2b615604e122ee932f4
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-an

In [ ]:
import pathlib, os
import time
import pandas as pd
import random
import requests
import json
import torch
import torch.multiprocessing as mp
from tqdm.notebook import tqdm
from tqdm.autonotebook import trange
from beir import util, LoggingHandler
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.datasets.data_loader import GenericDataLoader

In [ ]:
def load(dataset):
  hostname = 'localhost'
  index_name = dataset
  url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'.format(dataset)
  out_dir = os.path.join(os.getcwd(), 'datasets')
  data_path = util.download_and_unzip(url, out_dir)
  return GenericDataLoader(data_path).load(split='test') # 'test', 'train', 'dev'


In [ ]:
def retrieve(corpus, queries, qrels, model_name, batch_size, score_function):
  if batch_size is None:
    dr_model = EvaluateRetrieval(DRES(models.SentenceBERT(model_name)), score_function=score_function)
  else:
    dr_model = EvaluateRetrieval(
        DRES(models.SentenceBERT(model_name), batch_size=batch_size), score_function=score_function)
  res_model = dr_model.retrieve(corpus, queries)
  return dr_model.evaluate(qrels, res_model, dr_model.k_values)

In [ ]:
def evaluate(corpus, queries, qrels, model_name, batch_size, score_function):
  ndcg, _map, recall, precision = retrieve(corpus, queries, qrels, model_name, batch_size, score_function)
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

In [ ]:
sf_corpus, sf_queries, sf_qrels = load('scifact')
sf_als = evaluate(sf_corpus, sf_queries, sf_qrels, 'allenai-specter', 128, 'dot')
sf_amp = evaluate(sf_corpus, sf_queries, sf_qrels, 'all-mpnet-base-v2', 128, 'dot')
sf_adr = evaluate(sf_corpus, sf_queries, sf_qrels, 'all-distilroberta-v1', 128, 'dot')
sf_aml = evaluate(sf_corpus, sf_queries, sf_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
sf_mmp = evaluate(sf_corpus, sf_queries, sf_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
sf_mml = evaluate(sf_corpus, sf_queries, sf_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
sf_mdd = evaluate(sf_corpus, sf_queries, sf_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
sf_mdc = evaluate(sf_corpus, sf_queries, sf_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
sf_msdd = evaluate(sf_corpus, sf_queries, sf_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
sf_msdc = evaluate(sf_corpus, sf_queries, sf_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
sf_msbc = evaluate(sf_corpus, sf_queries, sf_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
sf_msra = evaluate(sf_corpus, sf_queries, sf_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')

  0%|          | 0/5183 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
sd_corpus, sd_queries, sd_qrels = load('scidocs')
sd_als = evaluate(sd_corpus, sd_queries, sd_qrels, 'allenai-specter', 128, 'dot')
sd_amp = evaluate(sd_corpus, sd_queries, sd_qrels, 'all-mpnet-base-v2', 128, 'dot')
sd_adr = evaluate(sd_corpus, sd_queries, sd_qrels, 'all-distilroberta-v1', 128, 'dot')
sd_aml = evaluate(sd_corpus, sd_queries, sd_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
sd_mmp = evaluate(sd_corpus, sd_queries, sd_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
sd_mml = evaluate(sd_corpus, sd_queries, sd_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
sd_mdd = evaluate(sd_corpus, sd_queries, sd_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
sd_mdc = evaluate(sd_corpus, sd_queries, sd_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
sd_msdd = evaluate(sd_corpus, sd_queries, sd_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
sd_msdc = evaluate(sd_corpus, sd_queries, sd_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
sd_msbc = evaluate(sd_corpus, sd_queries, sd_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
sd_msra = evaluate(sd_corpus, sd_queries, sd_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')

/content/datasets/scidocs.zip:   0%|          | 0.00/136M [00:00<?, ?iB/s]

  0%|          | 0/25657 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
nf_corpus, nf_queries, nf_qrels = load('nfcorpus')
nf_als = evaluate(nf_corpus, nf_queries, nf_qrels, 'allenai-specter', 128, 'dot')
nf_amp = evaluate(nf_corpus, nf_queries, nf_qrels, 'all-mpnet-base-v2', 128, 'dot')
nf_adr = evaluate(nf_corpus, nf_queries, nf_qrels, 'all-distilroberta-v1', 128, 'dot')
nf_aml = evaluate(nf_corpus, nf_queries, nf_qrels, 'all-MiniLM-L12-v2', 128, 'dot')
nf_mmp = evaluate(nf_corpus, nf_queries, nf_qrels, 'multi-qa-mpnet-base-dot-v1', 128, 'dot')
nf_mml = evaluate(nf_corpus, nf_queries, nf_qrels, 'multi-qa-MiniLM-L6-cos-v1', 128, 'cos_sim')
nf_mdd = evaluate(nf_corpus, nf_queries, nf_qrels, 'multi-qa-distilbert-dot-v1', 128, 'dot')
nf_mdc = evaluate(nf_corpus, nf_queries, nf_qrels, 'multi-qa-distilbert-cos-v1', 128, 'cos_sim')
nf_msdd = evaluate(nf_corpus, nf_queries, nf_qrels, 'msmarco-distilbert-base-tas-b', 128, 'dot')
nf_msdc = evaluate(nf_corpus, nf_queries, nf_qrels, 'msmarco-distilbert-dot-v5', 128, 'dot')
nf_msbc = evaluate(nf_corpus, nf_queries, nf_qrels, 'msmarco-bert-co-condensor', 128, 'dot')
nf_msra = evaluate(nf_corpus, nf_queries, nf_qrels, 'msmarco-roberta-base-ance-firstp', None, 'dot')

/content/datasets/nfcorpus.zip:   0%|          | 0.00/2.34M [00:00<?, ?iB/s]

  0%|          | 0/3633 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]



---



**SCIFACT** - [Homepage](https://allenai.org/data/scifact) - [Paper](https://arxiv.org/abs/2004.14974)

In [ ]:
pd.concat([sf_als, sf_amp, sf_adr, sf_aml, sf_mmp, sf_mml, sf_mdd, sf_mdc, sf_msdd, sf_msdc, sf_msbc, sf_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.44783,0.45681,0.45735,0.50667,0.54513,0.56055,0.07700,0.00977,0.00110,0.67861,0.85222,0.97500
all-mpnet-base-v2,0.60827,0.61595,0.61630,0.65570,0.68911,0.69653,0.08933,0.01070,0.00113,0.79011,0.94167,0.99667
all-distilroberta-v1,0.57580,0.58185,0.58226,0.63146,0.65978,0.66977,0.08900,0.01047,0.00113,0.78789,0.92000,0.99667
all-MiniLM-L12-v2,0.57186,0.57863,0.57901,0.62639,0.65549,0.66497,0.08833,0.01037,0.00111,0.78233,0.91167,0.98333
multi-qa-mpnet-base-dot-v1,0.54151,0.54938,0.55000,0.58907,0.62253,0.63794,0.08200,0.00993,0.00112,0.71756,0.86833,0.98667
multi-qa-MiniLM-L6-cos-v1,0.49919,0.50910,0.50955,0.54029,0.58909,0.60131,0.07433,0.01000,0.00111,0.65011,0.88033,0.97667
multi-qa-distilbert-dot-v1,0.49725,0.50593,0.50636,0.54839,0.59058,0.60085,0.07733,0.01007,0.00110,0.68761,0.88822,0.96667
multi-qa-distilbert-cos-v1,0.54929,0.55748,0.55783,0.59570,0.63221,0.64253,0.08167,0.01007,0.00110,0.72161,0.88533,0.96933
msmarco-distilbert-base-tas-b,0.59916,0.60459,0.60493,0.64276,0.66983,0.68106,0.08633,0.01013,0.00111,0.76150,0.89100,0.98333
msmarco-distilbert-dot-v5,0.55076,0.55811,0.55847,0.59490,0.62866,0.63907,0.08100,0.00993,0.00108,0.71483,0.87033,0.95333




---



**SCIDOCS** - [Homepage](https://allenai.org/data/scidocs) - [Paper](https://arxiv.org/abs/2004.07180)

In [ ]:
pd.concat([sd_als, sd_amp, sd_adr, sd_aml, sd_mmp, sd_mml, sd_mdd, sd_mdc, sd_msdd, sd_msdc, sd_msbc, sd_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.08170,0.10040,0.10433,0.14298,0.22616,0.29663,0.0737,0.01963,0.00366,0.14987,0.39858,0.74312
all-mpnet-base-v2,0.14426,0.17180,0.17580,0.23765,0.33664,0.39481,0.1269,0.02710,0.00409,0.25712,0.54997,0.82972
all-distilroberta-v1,0.13030,0.15713,0.16093,0.21694,0.31561,0.37487,0.1144,0.02564,0.00398,0.23192,0.52075,0.80807
all-MiniLM-L12-v2,0.13168,0.15766,0.16136,0.21818,0.31303,0.37175,0.1132,0.02495,0.00389,0.22932,0.50598,0.79053
multi-qa-mpnet-base-dot-v1,0.09721,0.11263,0.11523,0.16627,0.22982,0.28038,0.0839,0.01750,0.00297,0.17047,0.35492,0.60205
multi-qa-MiniLM-L6-cos-v1,0.09210,0.10629,0.10859,0.15780,0.21807,0.26593,0.0798,0.01667,0.00283,0.16170,0.33848,0.57345
multi-qa-distilbert-dot-v1,0.09098,0.10745,0.10991,0.15781,0.22602,0.27463,0.0804,0.01784,0.00296,0.16287,0.36245,0.60043
multi-qa-distilbert-cos-v1,0.09239,0.10910,0.11162,0.16004,0.22770,0.27644,0.0822,0.01796,0.00296,0.16690,0.36423,0.60232
msmarco-distilbert-base-tas-b,0.08543,0.10013,0.10248,0.14859,0.21063,0.25835,0.0748,0.01646,0.00280,0.15185,0.33462,0.56763
msmarco-distilbert-dot-v5,0.08077,0.09356,0.09566,0.14048,0.19639,0.24117,0.0710,0.01522,0.00260,0.14425,0.30865,0.52810




---



**NFCORPUS** - [Homepage](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/) - [Paper](https://www.cl.uni-heidelberg.de/~riezler/publications/papers/ECIR2016.pdf)

In [ ]:
pd.concat([nf_als, nf_amp, nf_adr, nf_aml, nf_mmp, nf_mml, nf_mdd, nf_mdc, nf_msdd, nf_msdc, nf_msbc, nf_msra])

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
allenai-specter,0.05045,0.06838,0.08021,0.18523,0.17471,0.27270,0.15201,0.05362,0.01867,0.08211,0.19321,0.53337
all-mpnet-base-v2,0.12034,0.15675,0.17180,0.33289,0.31496,0.40453,0.25294,0.08381,0.02126,0.16637,0.33853,0.66070
all-distilroberta-v1,0.09818,0.12841,0.14315,0.29239,0.27767,0.36836,0.22322,0.07715,0.02075,0.14700,0.29631,0.61755
all-MiniLM-L12-v2,0.11551,0.14938,0.16366,0.32250,0.30089,0.38860,0.24272,0.07963,0.02070,0.15468,0.31049,0.63148
multi-qa-mpnet-base-dot-v1,0.11234,0.14053,0.15333,0.31891,0.28629,0.37372,0.23746,0.07368,0.01981,0.15145,0.28526,0.60445
multi-qa-MiniLM-L6-cos-v1,0.10533,0.12970,0.14163,0.29674,0.26660,0.35088,0.21827,0.06780,0.01889,0.13996,0.26636,0.57816
multi-qa-distilbert-dot-v1,0.11721,0.14259,0.15485,0.31329,0.28154,0.36589,0.22786,0.06997,0.01946,0.15637,0.28252,0.58034
multi-qa-distilbert-cos-v1,0.10925,0.13540,0.14813,0.30249,0.27866,0.36503,0.22198,0.07111,0.01972,0.14791,0.28703,0.59908
msmarco-distilbert-base-tas-b,0.11945,0.14587,0.15814,0.31886,0.28662,0.37024,0.23003,0.07164,0.01938,0.15071,0.28012,0.58277
msmarco-distilbert-dot-v5,0.10836,0.13045,0.14121,0.29831,0.26471,0.34275,0.21796,0.06427,0.01816,0.14169,0.26435,0.53429


end of fun.